In [0]:
%sh

pip install pytrends

  Using cached pytrends-4.9.2-py3-none-any.whl (15 kB)
  Using cached lxml-4.9.3-cp39-cp39-manylinux_2_28_x86_64.whl (8.0 MB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-b6d2c43b-72ca-40cd-ab9c-8e3c4f0b2e7f/bin/python -m pip install --upgrade pip' command.


In [0]:
from pytrends.request import TrendReq
from pyspark.sql.functions import *


pytrends = TrendReq(hl='en-IN')
dbutils.widgets.text("country","india","Country")
ctry = dbutils.widgets.get("country")

print(ctry)

india


In [0]:
top_5_trend = pytrends.trending_searches(pn=f'{ctry}').head(10)
display(top_5_trend)

0
MLC
Man United vs Dortmund
Water
National Girlfriend Day
NTPC share price
Max Verstappen
Prime Minister Narendra Modi
Jasprit Bumrah
Vakkom Purushothaman
Lloyd Steel share price


In [0]:
spark_top_5 = spark.createDataFrame(top_5_trend)
top_5_ranked = spark_top_5.select("*")\
    .withColumnRenamed('0', 'search_term')\
    .withColumn("country", lit(f'{ctry}'))\
    .withColumn("rank", (monotonically_increasing_id() + 1).cast('int'))\
    .withColumn("date_as_of_date", current_date())
display(top_5_ranked)

search_term,country,rank,date_as_of_date
MLC,india,1,2023-07-31
Man United vs Dortmund,india,2,2023-07-31
Water,india,3,2023-07-31
National Girlfriend Day,india,4,2023-07-31
NTPC share price,india,5,2023-07-31
Max Verstappen,india,6,2023-07-31
Prime Minister Narendra Modi,india,7,2023-07-31
Jasprit Bumrah,india,8,2023-07-31
Vakkom Purushothaman,india,9,2023-07-31
Lloyd Steel share price,india,10,2023-07-31


In [0]:
top_5_ranked.describe

Out[6]: <bound method DataFrame.describe of DataFrame[search_term: string, country: string, rank: int, date_as_of_date: date]>

In [0]:
#top_5_ranked.write.mode('append').insertInto('trending_searches.countries_trending_searches')